# 1 - Générer les Embeddings
Cette cellule effectue l'importation des bibliothèques `os`, `numpy`, `pandas` et `SentenceTransformer`, puis charge le modèle de transformation de phrases `all-MiniLM-L6-v2`. Elle lit ensuite le fichier `texte-nettoye.txt` et segmente son contenu en phrases en utilisant le point comme séparateur. Après avoir supprimé les espaces inutiles et filtré les segments vides, elle génère des embeddings vectoriels pour chaque phrase à l’aide du modèle chargé. Les résultats sont ensuite stockés dans un DataFrame associant chaque phrase à son vecteur d’embedding, puis enregistrés dans un fichier CSV intitulé `embeddings.csv`. C'est le fichier que nous allons exploiter.


In [7]:
import os
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

with open("texte-nettoye.txt", "r", encoding="utf-8") as f:
    texte = f.read()

chunks = texte.split(".")  
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

embeddings = model.encode(chunks)

df = pd.DataFrame({
    "chunk": chunks,
    "embedding": embeddings.tolist()
})

df.to_csv("embeddings.csv", index=False)

print("Embeddings enregistrés dans embeddings.csv :)")


Embeddings enregistrés dans embeddings.csv :)


# 2 - Indexation FAISS 

Ici, nous utilisons la bibliothèque `faiss` pour optimiser la recherche de similarité entre les embeddings. Ensuite, nous lisons le fichier `embeddings.csv` pour extraire les segments de texte ainsi que leurs embeddings, que nous convertissons en un tableau NumPy de type `float32` (format attendu par `faiss`). Nous créons un index FAISS de type `IndexFlatL2`, basé sur la distance euclidienne (L2), et nous y insérons les embeddings. Enfin, cet index est sauvegardé dans un fichier `faiss_index.bin`, ce qui nous permettra d'effectuer des recherches rapides sur les phrases du corpus.



In [2]:
import faiss

df = pd.read_csv("embeddings.csv")
chunks = df["chunk"].tolist()
embeddings = np.array(df["embedding"].apply(eval).tolist()).astype('float32')

index = faiss.IndexFlatL2(embeddings.shape[1])  
index.add(embeddings)

faiss.write_index(index, "faiss_index.bin")

print("Index FAISS sauvegardé dans faiss_index.bin")


Index FAISS sauvegardé dans faiss_index.bin


# 3 - Recherche de similarité

Nous chargeons uniquement la colonne `chunk` du fichier `embeddings.csv`, qui contient les segments de texte et nous récupérons l'index à partir du fichier `faiss_index.bin` pour effectuer des recherches directement sur les vecteurs enregistrés.
La fonction `rechercher_similaires(query, top_k=5)` encode la requête en un vecteur `float32` et le compare aux vecteurs de l’index FAISS à l’aide de `.search()`, qui renvoie les indices des `top_k` segments les plus proches ainsi que leurs distances.
Enfin, nous testons le système avec la requête **"Quel est le lien entre IA et philosophie ?"**. La fonction retourne les cinq segments les plus similaires avec leurs scores de distance, un score plus bas indiquant une plus grande proximité avec la requête.




In [8]:
import faiss
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

chunks = pd.read_csv("embeddings.csv", usecols=["chunk"])["chunk"].tolist()

index = faiss.read_index("faiss_index.bin")

def rechercher_similaires(query, top_k=5):
    query_embedding = model.encode([query]).astype('float32')  
    distances, indices = index.search(query_embedding, top_k)  
    
    resultats = [(chunks[idx], distances[0][i]) for i, idx in enumerate(indices[0]) if idx != -1]
    return resultats

query = "Quel est le lien entre IA et philosophie ?"
resultats = rechercher_similaires(query)

for texte, score in resultats:
    print(f"Score: {score:.4f} - Texte: {texte}")


Score: 1.0322 - Texte: Défis et Éthique

Malgré ses nombreux avantages, l'IA pose également des défis importants
Score: 1.1991 - Texte: En comprenant son historique, ses applications, et ses défis, nous pouvons mieux appréhender son potentiel et ses implications futures
Score: 1.2376 - Texte: Dans la finance, elle est utilisée pour détecter les fraudes et optimiser les investissements
Score: 1.2548 - Texte: Applications de l'IA

L'IA est aujourd'hui utilisée dans divers secteurs, tels que la santé, la finance, et le divertissement
Score: 1.2887 - Texte: Conclusion

L'intelligence artificielle est un domaine en pleine expansion qui promet de transformer notre monde de manière significative


# 4 - Génération d'une réponse avec un LLM (OpenAI)

## 🔍 4.1. Recherche des passages similaires  
La fonction `rechercher_similaires(query, top_k=5)` permet de retrouver les segments de texte les plus proches d’une requête donnée :  
- La requête est convertie en embedding à l’aide du modèle `SentenceTransformer`.  
- L’index FAISS est utilisé pour rechercher les `top_k` passages les plus similaires en fonction de la distance.  
- La fonction retourne une liste des segments correspondants avec leurs scores de similarité.  

## 📝 4.2. Recherche et génération de réponse  
- L’utilisateur pose une question, ici **"cite le passage relatif à l'éthique et son titre ?"**.  
- La fonction `rechercher_similaires()` extrait les passages pertinents du corpus.  
- Ces passages sont combinés en un **contexte textuel**.  
- OpenAI (`gpt-3.5-turbo`) génère une réponse à partir de ce contexte.  
- La réponse est affichée à l’utilisateur.  

🚀 **Ce système permet d’extraire rapidement des passages pertinents et de générer des réponses précises à partir d’un texte.**


In [6]:
import openai
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = openai.OpenAI(api_key=OPENAI_API_KEY)

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

chunks = pd.read_csv("embeddings.csv", usecols=["chunk"])["chunk"].tolist()

index = faiss.read_index("faiss_index.bin")

def rechercher_similaires(query, top_k=5):
    query_embedding = model.encode([query]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    
    resultats = [(chunks[idx], distances[0][i]) for i, idx in enumerate(indices[0]) if idx != -1]
    return resultats

def generer_reponse(context, question):
    prompt = f"Voici des extraits pertinents : {context}\n\nRéponds à cette question : {question}"
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Tu es un assistant utile."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

query = "cite le passage relatif à l'éthique et son titre ?"
resultats = rechercher_similaires(query)

context = "\n".join([texte for texte, _ in resultats])

reponse = generer_reponse(context, query)
print("Réponse générée :")
print(reponse)

Réponse générée :
Le passage relatif à l'éthique dans les extraits que vous avez fournis est le suivant :
"Il est crucial de développer des cadres éthiques pour guider l'utilisation responsable de l'IA"
Ce passage se situe dans la section intitulée "Défis et Éthique".
